In [ ]:
sudo apt-get install git
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
sudo apt-get install python3-pip
pip3 install -r requirements.txt

import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

def create_validation_set(train_path, val_path, split_ratio=0.3):
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    train_images = os.listdir(os.path.join(train_path, 'images'))
    train_images = [f for f in train_images if f.endswith(('.jpg', '.jpeg', '.png'))]

    _, val_images = train_test_split(train_images, test_size=split_ratio, random_state=42)

    for image_file in val_images:
        src_image = os.path.join(train_path, 'images', image_file)
        dst_image = os.path.join(val_path, 'images', image_file)
        shutil.copy2(src_image, dst_image)

        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_label = os.path.join(train_path, 'labels', label_file)
        dst_label = os.path.join(val_path, 'labels', label_file)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"Created validation set with {len(val_images)} images")

def _preproc(image, output_height=512, output_width=512, resize_side=512):
    ''' imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px'''
    h, w = image.shape[0], image.shape[1]
    scale = resize_side / min(h, w)
    resized_image = np.array(Image.fromarray(image).resize((int(w*scale), int(h*scale))))
    cropped_image = resized_image[:output_height, :output_width]
    return cropped_image

def Create_npy(imagespath, imgsize, ext):
    images_list = [img_name for img_name in os.listdir(imagespath) if os.path.splitext(img_name)[1].lower() == '.'+ext.lower()]
    calib_dataset = np.zeros((len(images_list), imgsize, imgsize, 3), dtype=np.float32)

    for idx, img_name in enumerate(sorted(images_list)):
        img_path = os.path.join(imagespath, img_name)
        try:
            img = Image.open(img_path)
            img = img.convert("RGB")
            img_np = np.array(img)

            img_preproc = _preproc(img_np, imgsize, imgsize, imgsize)
            calib_dataset[idx, :, :, :] = img_preproc
            print(f"Processed image {img_path}")
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    np.save('calib_set.npy', calib_dataset)

train_path = '/home/amap/yolov5/Train'
val_path = '/home/amap/yolov5/Val'

create_validation_set(train_path, val_path)

Create_npy('/home/amap/yolov5/Train/images', 512, 'jpg')

!python3 train.py --img 512 --batch 16 --epochs 300 --data /home/amap/yolov5/data.yaml --weights yolov5n.pt --cache

!python3 detect.py --weights /home/amap/yolov5/runs/train/PedestrianLight/weights/best.pt --source /home/amap/yolov5/video/vd.mp4
